In [ ]:
from sepal_ui import mapping as sm

Map = sm.SepalMap()
Map.add_control(sm.FullScreenControl(Map))
Map

In [ ]:
import ee

ee.Initialize()

In [ ]:
france = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(
    ee.Filter.eq("ADM0_NAME", "France")
)
Map.addLayer(france, {"color": "green"}, "france")
Map.centerObject(france);

In [ ]:
bb = france.geometry().bounds().transform(ee.Projection("EPSG:3857"), maxError=1)
Map.addLayer(bb, {"color": "blue"}, "bb")

In [ ]:
coords = ee.List(bb.coordinates().get(0))
xmin = ee.List(coords.get(0)).get(0)
ymin = ee.List(coords.get(0)).get(1)
xmax = ee.List(coords.get(2)).get(0)
ymax = ee.List(coords.get(2)).get(1)

In [ ]:
size = 100000
xx = ee.List.sequence(xmin, ee.Number(xmax), size)
yy = ee.List.sequence(ymin, ee.Number(ymax), size)

In [ ]:
def map_overver_x(x):
    def map_over_y(y):
        xmin_loc = ee.Number(x)
        ymin_loc = ee.Number(y)
        xmax_loc = xmin_loc.add(ee.Number(size))
        ymax_loc = ymin_loc.add(ee.Number(size))

        coords = ee.List([xmin_loc, ymin_loc, xmax_loc, ymax_loc])
        rect = ee.Geometry.Rectangle(coords, "EPSG:3857", False)

        return ee.Feature(rect)

    return yy.map(map_over_y)

In [ ]:
cells = xx.map(map_overver_x).flatten()

In [ ]:
bins = ee.FeatureCollection(cells).filterBounds(france)
Map.addLayer(bins, {"color": "gray"}, "bins")

In [ ]:
from sepal_ui import scripts
import inspect

inspect.getfile(scripts)

In [ ]:
import inspect
from sepal_ui.scripts import utils as su
import pandas as pd
from pathlib import Path
import ee

ee.Initialize()

admin = 816
level = 0
admin_dataset = ee.FeatureCollection(f"FAO/GAUL/2015/level{level}")
gaul_dataset = Path(inspect.getfile(su)).parent / "gaul_database.csv"
df = pd.read_csv(gaul_dataset)
is_in = df.filter([f"ADM{i}_CODE" for i in range(3)]).isin([admin])
aoi_level = int(is_in[~((~is_in).all(axis=1))].idxmax(1).iloc[0][3])
line = df[~((~is_in).all(axis=1))]
requested_level_code = int(line[f"ADM{level}_CODE"].values[0])

# use the computerd bin level only when requested level is bigger than aoi
bin_level = level if aoi_level > level else aoi_level
bin_code = requested_level_code if aoi_level > level else aoi_model.admin

bins = admin_dataset.filter(ee.Filter.eq(f"ADM{bin_level}_CODE", bin_code))

In [ ]:
bin_level

In [ ]:
bin_code